In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),  # Input size is 2 (latent space)
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2)  # Output size should be 2 for 2D data
        )

    def forward(self, input):
        return self.main(input)

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),  # Input size should match the generated data size
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 1)  # Output size is 1 for binary classification
        )

    def forward(self, input):
        return self.main(input)

# Create instances of the generator and discriminator and move to device
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Optimizers
gen_optimizer = optim.RMSprop(generator.parameters(), lr=0.001)
disc_optimizer = optim.RMSprop(discriminator.parameters(), lr=0.001)

# Function to generate noise
def sample_Z(m, n):
    return torch.Tensor(np.random.uniform(-1., 1., size=[m, n])).to(device)

# Load training data from m_hist.txt
def load_data(file_path):
    # Load only the first two columns to avoid dimension mismatch
    data = np.loadtxt(file_path, usecols=[0, 1])  
    return torch.Tensor(data).to(device)

# Load your real training data
real_data_file = '/home/jovyan/plots-gan/basic-gans/code/m_hist.txt'  # Full path to your data file
x_plot = load_data(real_data_file)

# Training parameters
batch_size = min(256, len(x_plot))  # Adjust batch size to be smaller than or equal to the number of data points
nd_steps = 10
ng_steps = 10
num_iterations = 12001  # Set to 12000 iterations

# Path to the loss log file
loss_log_file = '../plots/loss_log.csv'

# Write the header of the CSV file
with open(loss_log_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Iteration', 'Discriminator Loss', 'Generator Loss'])

# Training loop
for i in range(num_iterations):
    for _ in range(nd_steps):
        # Sample a batch of real data
        indices = np.random.choice(len(x_plot), batch_size, replace=False)
        X_batch = x_plot[indices]
        Z_batch = sample_Z(batch_size, 2)

        # Real data labels are 1, generated data labels are 0
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Compute discriminator loss on real data
        real_outputs = discriminator(X_batch)
        real_loss = criterion(real_outputs, real_labels)

        # Compute discriminator loss on fake data
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        # Total discriminator loss
        disc_loss = real_loss + fake_loss

        # Backpropagation and optimization for discriminator
        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()

    for _ in range(ng_steps):
        Z_batch = sample_Z(batch_size, 2)
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data)

        # Generator loss
        gen_loss = criterion(fake_outputs, real_labels)

        # Backpropagation and optimization for generator
        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()

    # Save the losses to the CSV file
    with open(loss_log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([i, disc_loss.item(), gen_loss.item()])

    print(f"Iterations: {i}\t Discriminator loss: {disc_loss.item():.4f}\t Generator loss: {gen_loss.item():.4f}")

    # Plotting every 1000 iterations
    if i % 1000 == 0:
        plt.figure()
        with torch.no_grad():
            g_plot = generator(sample_Z(batch_size, 2)).cpu().numpy()  # Generate new data for plotting
        plt.scatter(x_plot.cpu()[:, 0], x_plot.cpu()[:, 1], color='blue', label='Real Data', alpha=0.6)
        plt.scatter(g_plot[:, 0], g_plot[:, 1], color='orange', label='Generated Data', alpha=0.6)

        plt.legend()
        plt.title(f'Samples at Iteration {i}', fontsize=14)
        plt.xlabel('X-axis Label', fontsize=12)
        plt.ylabel('Y-axis Label', fontsize=12)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'../plots/iterations/iteration_{i}.png', dpi=300)
        plt.close()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),  # Input size is 2 (latent space)
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2)  # Output size should be 2 for 2D data
        )

    def forward(self, input):
        return self.main(input)

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),  # Input size should match the generated data size
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 1)  # Output size is 1 for binary classification
        )

    def forward(self, input):
        return self.main(input)

# Create instances of the generator and discriminator and move to device
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Optimizers
gen_optimizer = optim.RMSprop(generator.parameters(), lr=0.001)
disc_optimizer = optim.RMSprop(discriminator.parameters(), lr=0.001)

# Function to generate noise
def sample_Z(m, n):
    return torch.Tensor(np.random.uniform(-1., 1., size=[m, n])).to(device)

# Load training data from m_hist.txt
def load_data(file_path):
    # Load only the first two columns to avoid dimension mismatch
    data = np.loadtxt(file_path, usecols=[0, 1])  
    return torch.Tensor(data).to(device)

# Load your real training data
real_data_file = '/home/jovyan/plots-gan/basic-gans/code/m_hist.txt'  # Full path to your data file
x_plot = load_data(real_data_file)

# Training parameters
batch_size = min(256, len(x_plot))  # Adjust batch size to be smaller than or equal to the number of data points
nd_steps = 10
ng_steps = 10
num_iterations = 12001  # Set to 12000 iterations

# Path to the loss log file
loss_log_file = '../plots/loss_log.csv'

# Create a directory for saving checkpoints if it doesn't exist
checkpoint_dir = '../plots/checkpoint/'
os.makedirs(checkpoint_dir, exist_ok=True)

# Function to save model checkpoint
def save_checkpoint(iteration):
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_{iteration}.tar")
    torch.save({
        'iteration': iteration,
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'gen_optimizer_state_dict': gen_optimizer.state_dict(),
        'disc_optimizer_state_dict': disc_optimizer.state_dict(),
    }, checkpoint_path)
    print(f"Checkpoint saved at iteration {iteration}")

# Function to load model checkpoint
def load_checkpoint():
    # Find the latest checkpoint
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.tar')]
    if not checkpoint_files:
        print("No checkpoints found.")
        return 0  # No checkpoints found, start from iteration 0

    latest_checkpoint = max(checkpoint_files, key=lambda f: int(f.split('_')[1].split('.')[0]))
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
    
    checkpoint = torch.load(checkpoint_path)
    generator.load_state_dict(checkpoint['generator_state_dict'])
    discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
    gen_optimizer.load_state_dict(checkpoint['gen_optimizer_state_dict'])
    disc_optimizer.load_state_dict(checkpoint['disc_optimizer_state_dict'])
    print(f"Checkpoint loaded from iteration {checkpoint['iteration']}")
    return checkpoint['iteration']

# Load the last checkpoint if exists
start_iteration = load_checkpoint()

# Write the header of the CSV file if starting fresh
if start_iteration == 0:
    with open(loss_log_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Iteration', 'Discriminator Loss', 'Generator Loss'])

# Training loop
for i in range(start_iteration, num_iterations):
    for _ in range(nd_steps):
        # Sample a batch of real data
        indices = np.random.choice(len(x_plot), batch_size, replace=False)
        X_batch = x_plot[indices]
        Z_batch = sample_Z(batch_size, 2)

        # Real data labels are 1, generated data labels are 0
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Compute discriminator loss on real data
        real_outputs = discriminator(X_batch)
        real_loss = criterion(real_outputs, real_labels)

        # Compute discriminator loss on fake data
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        # Total discriminator loss
        disc_loss = real_loss + fake_loss

        # Backpropagation and optimization for discriminator
        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()

    for _ in range(ng_steps):
        Z_batch = sample_Z(batch_size, 2)
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data)

        # Generator loss
        gen_loss = criterion(fake_outputs, real_labels)

        # Backpropagation and optimization for generator
        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()

    # Save the losses to the CSV file
    with open(loss_log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([i, disc_loss.item(), gen_loss.item()])

    print(f"Iterations: {i}\t Discriminator loss: {disc_loss.item():.4f}\t Generator loss: {gen_loss.item():.4f}")

    # Save models and optimizers as tar files every 1000 iterations
    if i % 1000 == 0 and i != start_iteration:
        save_checkpoint(i)

    # Plotting every 1000 iterations
    if i % 1000 == 0:
        plt.figure()
        with torch.no_grad():
            g_plot = generator(sample_Z(batch_size, 2)).cpu().numpy()  # Generate new data for plotting
        plt.scatter(x_plot.cpu()[:, 0], x_plot.cpu()[:, 1], color='blue', label='Real Data', alpha=0.6)
        plt.scatter(g_plot[:, 0], g_plot[:, 1], color='orange', label='Generated Data', alpha=0.6)

        plt.legend()
        plt.title(f'Samples at Iteration {i}', fontsize=14)
        plt.xlabel('X-axis Label', fontsize=12)
        plt.ylabel('Y-axis Label', fontsize=12)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'../plots/iterations/iteration_{i}.png', dpi=300)
        plt.close()
